# DATASET >> https://universe.roboflow.com/force-iaope/cars-egrgd/dataset/1#

# PRÉ-AMBULO - IMPORT LIBS

In [ ]:
from ultralytics import YOLO
from threading import Thread
from PIL import Image
import queue as q
import shutil
import cv2
import os

from modules.helper.helper import create_video_from_images, get_framerate

import torch
import torchvision
print("torch:", torch.__version__)
print("torchvision:", torchvision.__version__)
print("CUDA disponível:", torch.cuda.is_available())

# https://docs.ultralytics.com/datasets/

# TRAIN

In [ ]:
# # Create a new YOLO model from scratch
model = YOLO("yolo11n.yaml")

# Load a pretrained YOLO model (recommended for training)
model = YOLO("yolo11n.pt")

# Train the model using the 'coco8.yaml' dataset for 3 epochs # c:\dev\datasets
results = model.train(
    data="data_train.yaml",        # Arquivo de configuração do seu dataset
    epochs=1000,                   # Número de épocas (pode ser ajustado conforme necessário)
    device="cuda" if torch.cuda.is_available() else "cpu",  # Verifica se há GPU disponível, caso contrário usa CPU
    workers=2,                     # Número de workers para carregamento dos dados (aumentado para agilizar)
    imgsz=512,                     # Tamanho das imagens (960x960)
    batch=4,                       # Tamanho do batch (aumentado para melhor estabilidade, se sua GPU suportar)
    cache="disk",                  # Utiliza cache no disco para garantir resultados determinísticos
    save_period=5,                 # Salva o modelo a cada 5 épocas
    name="yolo11n_512_1000_max",   # Nome da pasta de treinamento
    exist_ok=True,                 # Sobrescreve o diretório existente, se houver
    project="../../runs/train",    # Diretório onde os resultados serão salvos
    resume=False,                  # Não retoma de um checkpoint anterior
    patience=10,                   # Aumenta a paciência do EarlyStopping para 50 épocas (ajuste conforme necessário)
    optimizer="AdamW",             # Otimizador SGD Adam AdamW pode melhorar a convergência
    pretrained=True,               # Utiliza pesos pré-treinados
    optimize=True,                 # Otimiza o modelo para melhor desempenho
    classes=[1, 2, 3, 4],          # Escolhe as classes a serem treinadas (0: pessoas, 1: carros)
    # hyp="data/hyp.scratch-low.yaml" # Arquivo de hiperparâmetros; ajuste-o conforme o seu dataset
)

# Validate the model on the validation set
valid = model.val()

# Export the model to ONNX format
success = model.export(format="onnx", dynamic=True)


# TEST IN VIDEO

In [ ]:
# Load the YOLO11 model
model           = YOLO("..\\..\\runs\\train\\yolo11n_512_1000_max\\weights\\best.pt")

# Open the video file
video_path      = "input\\OneDrive\\Ponto 1\\Av. Mato Grosso x BR153 (Goiânia)\\Manhã\\Rec95_20250318065523_S_1.avi" # "input\\Teste.mp4"
cap = cv2.VideoCapture(video_path)

# Path to save the output frames
save            = "output\\detect"
shutil.rmtree(save)
os.makedirs(save, exist_ok=True)

# Initialize the count frame index and queue 
idx = 0
queue = q.Queue(maxsize=100)  # Create a queue with a maximum size of 500

# Function to save the frame as an image
def SaveFrame():
    while True:
        frame, idx = queue.get()  # Get the frame and index from the queue
        if frame is None or idx is None: break
        cv2.imwrite(os.path.join(save, f"frame_{idx}.jpg"), frame)

# Create and start threads for saving frames
SaveFrameThreads = []
for i in range(3):
    thread = Thread(target=SaveFrame, daemon=True)  # Create a thread to save frames
    thread.start()  # Start the thread
    SaveFrameThreads.append(thread)  # Add the thread to the list

# Loop through the video frames
while cap.isOpened():
    
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        
        try:
            # Run YOLO11 tracking on the frame, persisting tracks between frames
            results = model.track(
                frame, 
                persist=True,
                classes=[1, 2, 3, 4],   # Classes to track
                conf=0.5,               # Confidence threshold
                iou=0.45,               # IoU threshold
                device= (
                    "cuda" 
                    if torch.cuda.is_available() 
                    else 
                    "cpu"
                ),                      # Verifica se há GPU disponível, caso contrário usa CPU
                show=False,             # Show the results
                save=False,             # Save the results
                save_txt=False,         # Save results to text files
                save_conf=False,        # Save confidence scores
                save_crop=False,        # Save cropped images of detected objects
                # save_dir=save,        # Directory to save results
                # project=save,         # Project name
                name="detect",          # Name of the folder to save results
                exist_ok=True,          # Overwrite existing results
                line_width=1,           # Line thickness for bounding boxes
                show_labels=True,       # Hide labels on bounding boxes
                show_conf=True,         # Hide confidence scores on bounding boxes
                half=True,              # Use half precision (FP16) for inference
                dnn=False,              # Use OpenCV DNN for inference
                vid_stride=1,           # Frame stride for video processing
                agnostic_nms=False,     # Use class-agnostic NMS
                augment=False,          # Use augmentation during inference
                visualize=False,        # Visualize the results
                verbose=False,          # Verbose output
            )
            
            # Visualize the results on the frame
            annotated_frame = results[0].plot()
            if annotated_frame is None:
                print(f"Resultado {idx} não contém dados válidos para plotar.")
                continue
            
            im_rgb = Image.fromarray(annotated_frame[..., ::-1])  # RGB-order PIL image
            queue.put((annotated_frame, idx))  # Add the frame to the queue for saving
            idx += 1  # Increment the frame index
        except ValueError as e:
            print(f"Erro ao processar o resultado {idx}: {e}")
        except Exception as e:
            print(f"Erro inesperado ao processar o resultado {idx}: {e}")

    else:
        # Break the loop if the end of the video is reached
        break

# Wait for all threads to finish processing
for idx, thread in enumerate(SaveFrameThreads):
    print(f"Wait for the thread to finish {idx}")
    queue.put((None, None))
    thread.join()  # Wait for all threads to finish

# Release the video capture object and close the display window
cap.release()

# Exemplo de uso
image_folder = save
output_video_path = "output_video.mp4"
frame_rate = get_framerate(video_path)  # Substitua pelo frame rate desejado
create_video_from_images(image_folder, output_video_path, frame_rate)

# Abrir vídeo automaticamente após criação
os.system(f"start {output_video_path}")

Wait for the thread to finish 1
Wait for the thread to finish 2
